# 步骤 1: 准备工作与环境设置

**目标:** 导入所有需要的库，并设置好文件路径和全局变量。

In [1]:
# --- 导入库与全局配置 ---
import json
import re
import html
import unicodedata
import time
import pandas as pd
import spacy
from tqdm import tqdm

# --- 新增的库，用于优化 ---
# Flashtext 用于高效的多关键词搜索，替代正则表达式
from flashtext import KeywordProcessor
# concurrent.futures 和 multiprocessing 用于实现并行处理
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp

# 让 tqdm 与 pandas 的 apply 配合使用
tqdm.pandas()

# --- 全局文件与参数配置 ---
#ALIYUN_OSS_PATH = '/mnt/aliyun_oss/'
ALIYUN_OSS_PATH = ''
KEYWORD_JSON_PATH = ALIYUN_OSS_PATH + 'data_raw/china_keywords_collection.json'
SOURCE_NEWS_FILE = ALIYUN_OSS_PATH + 'data_raw/final_merged_all_news.csv'
CANDIDATES_FILE = ALIYUN_OSS_PATH + 'data_processed/china_news_candidates.csv'
FINAL_RESULT_FILE = ALIYUN_OSS_PATH + 'data_processed/final_china_news.csv'
REJECTED_FILE = ALIYUN_OSS_PATH + 'data_processed/china_news_rejected_articles.csv'

# 新闻列的列名
NEWS_COLUMN = 'CONTENT'

# 分块处理时每个块的大小
CHUNKSIZE = 10000

# --- 新增：并行处理配置 ---
# 使用 CPU 核心数减 1，留一个核心给系统，避免卡顿
NUM_PROCESSES = mp.cpu_count() - 1 if mp.cpu_count() > 1 else 1

print(f"✅ 块 1: 库导入和配置完成。将使用 {NUM_PROCESSES} 个进程进行并行处理。")

✅ 块 1: 库导入和配置完成。将使用 11 个进程进行并行处理。


# 步骤 2: 初筛准备 - 构建智能正则表达式

**目标:** 读取 中国相关关键词 JSON 文件，并执行我们讨论过的所有逻辑来构建一个强大、高效的正则表达式。

In [2]:
# --- 构建初筛用的 Flashtext 关键词处理器 ---

def build_keyword_processor(json_path):
    """
    从关键词 JSON 文件中构建一个高效的 Flashtext KeywordProcessor。
    """
    print(f"正在从 {json_path} 加载关键词...")
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            keywords_data = json.load(f)
    except FileNotFoundError:
        print(f"❌ 错误: 关键词文件未找到 {json_path}")
        return None, None

    print(f"共加载 {len(keywords_data)} 个关键词对象。")

    # 1. 提取全部别名
    all_aliases = set()
    for item in keywords_data:
        all_aliases.add(item['keyword'])
        for alias in item.get('aliases', []):
            all_aliases.add(alias)

    print(f"提取出 {len(all_aliases)} 个不重复的关键词/别名。")

    # 2. 初始化 Flashtext 处理器
    # case_sensitive=False 使其不区分大小写，等同于原正则的 (?i)
    keyword_processor = KeywordProcessor(case_sensitive=False)

    # 3. 将所有关键词添加到处理器中
    # Flashtext 内部会构建优化的数据结构 (Aho-Corasick automaton)
    for kw in all_aliases:
        keyword_processor.add_keyword(kw)

    print("✅ 高效关键词处理器 (Flashtext) 构建完成。")

    return keyword_processor, keywords_data


# 执行构建
keyword_processor, keywords_data = build_keyword_processor(KEYWORD_JSON_PATH)

print("\n✅ 块 2: 初筛准备工作完成。")

正在从 data_raw/china_keywords_collection.json 加载关键词...
共加载 274 个关键词对象。
提取出 394 个不重复的关键词/别名。
✅ 高效关键词处理器 (Flashtext) 构建完成。

✅ 块 2: 初筛准备工作完成。


# 步骤 3: 执行阶段一 - 调用外部脚本进行快速初筛

**目标:** 对大文件进行分块扫描，应用正则表达式，并保存候选集。这将是整个流程中最耗时的部分。

In [3]:
# --- 步骤 3: 执行阶段一 - 调用外部脚本进行并行初筛 (优化版) ---
import subprocess
import sys

print("--- 阶段一: 调用外部脚本进行并行初筛 ---")
start_time_script = time.time()

# 构建要执行的命令
# 我们将Python解释器的路径、脚本名和所有必需的文件路径作为参数传递
command = [
    sys.executable,  # 使用当前Jupyter环境的Python解释器，保证环境一致
    '_parallel_filter.py',
    SOURCE_NEWS_FILE,
    KEYWORD_JSON_PATH,
    CANDIDATES_FILE
]

print(f"将要执行的命令: {' '.join(command)}")

try:
    # 执行脚本
    # check=True: 如果脚本返回非零退出码（即出错），则会抛出异常
    # capture_output=True: 捕获脚本的 stdout 和 stderr
    # text=True: 将捕获的输出解码为文本
    result = subprocess.run(
        command,
        check=True,
        capture_output=True,
        text=True,
        encoding='utf-8' # 明确编码以避免乱码
    )

    # 打印脚本的实时输出
    print("\n--- 外部脚本输出 ---")
    print(result.stdout)
    print("---------------------\n")

    end_time_script = time.time()
    print(f"✅ 外部脚本执行成功！")
    print(f"阶段一总耗时: {(end_time_script - start_time_script) / 60:.2f} 分钟。")

except subprocess.CalledProcessError as e:
    # 如果脚本执行失败，打印详细的错误信息
    print("\n❌ 外部脚本执行失败！")
    print(f"返回码: {e.returncode}")
    print("\n--- STDOUT (标准输出) ---")
    print(e.stdout)
    print("\n--- STDERR (错误输出) ---")
    print(e.stderr)
    print("--------------------------\n")
    print("请检查上面的错误信息。")

print("✅ 块 3: 初筛流程执行完毕。")

--- 阶段一: 调用外部脚本进行并行初筛 ---
将要执行的命令: C:\Users\liuma\anaconda3\envs\LEO_WSJ_China\python.exe parallel_filter.py data_raw/final_merged_all_news.csv data_raw/china_keywords_collection.json data_processed/china_news_candidates.csv

--- 外部脚本输出 ---
--- 开始执行并行初筛脚本 ---
✅ 关键词处理器构建完成，包含 394 个关键词。
正在计算文件总块数...
文件约包含 377 个数据块。开始处理...

--- 初筛流程执行完毕 ---
总共处理了 150 个数据块。
总共找到 179380 篇候选文章。
结果已保存到: data_processed/china_news_candidates.csv
耗时: 2.54 分钟。

---------------------

✅ 外部脚本执行成功！
阶段一总耗时: 2.55 分钟。
✅ 块 3: 初筛流程执行完毕。


# 步骤 4: 精筛准备 - 加载模型与定义规则

**目标:** 负责加载 spaCy 模型和数据，并定义所有用于精筛的“否决规则”函数。

In [4]:
# --- 精筛准备 - 加载模型与定义否决规则 ---
print("--- 阶段二准备: 加载 spaCy 模型 ---")
# 禁用不需要的组件可以加快加载速度和减少内存占用
nlp = spacy.load("en_core_web_lg", disable=["ner", "lemmatizer", "attribute_ruler"])
print(f"✅ spaCy 模型 '{nlp.meta['name']}' 的基础组件加载成功。")

# --- 准备 PhraseMatcher ---
print("正在准备 PhraseMatcher...")
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

keyword_lookup = {}
patterns = []

for item in keywords_data:
    # 为每个别名创建 Doc 对象作为 pattern
    for alias in item.get('aliases', [item['keyword']]):
        # 使用 nlp.make_doc 确保 pattern 是 spaCy 的 Doc 对象
        patterns.append(nlp.make_doc(alias))
        # 建立一个小写的 alias -> item 信息的查找表
        keyword_lookup[alias.lower()] = {
            'type': item.get('type'),
            'category': item.get('category'),
            'tier': item.get('relevance_tier')
        }

# 一次性将所有 patterns 加入 matcher，效率更高
matcher.add("ChinaKeywords", patterns)
print(f"✅ PhraseMatcher 准备完成，已添加 {len(patterns)} 个模式。")


# --- 定义否决规则函数 (保持不变) ---
def check_negation(doc, keywords_in_doc):
    for token in doc:
        if token.dep_ == "neg":
            if token.head.text.lower() in keywords_in_doc:
                return True, f"否定语境: '{token.text}' 修饰了关键词 '{token.head.text}'"
    return False, ""


def check_hypothetical(doc, keywords_in_doc):
    for sent in doc.sents:
        if sent.root.text.lower() == 'if' or sent[0].text.lower() in ['if', 'unless']:
            for token in sent:
                if token.text.lower() in keywords_in_doc:
                    return True, f"假设语境: 句子以 '{sent[0].text}' 开头"
    return False, ""


def check_low_tier_only(found_keywords_info):
    if not found_keywords_info:
        return True, "未找到任何关键词"
    tiers = [info['tier'] for info in found_keywords_info]
    if all(tier <= 2 for tier in tiers):
        strong_categories = {"Politics", "Economics", "Geopolitics", "Technology", "Finance", "Military"}
        categories = {info['category'] for info in found_keywords_info}
        if not strong_categories.intersection(categories):
            return True, "只包含Tier 1/2的弱相关关键词 (如文化、体育)"
    return False, ""


print("✅ 块 4: 精筛规则定义和 Matcher 准备完成。")


--- 阶段二准备: 加载 spaCy 模型 ---
✅ spaCy 模型 'core_web_lg' 的基础组件加载成功。
正在准备 PhraseMatcher...
✅ PhraseMatcher 准备完成，已添加 397 个模式。
✅ 块 4: 精筛规则定义和 Matcher 准备完成。


# 步骤 5: 执行阶段二 - 精筛流程

**目标:** 加载候选集，应用所有否决规则，然后保存最终结果和被拒绝的文章。

In [5]:
# --- 执行阶段二 - 使用 PhraseMatcher 进行高效精筛 ---

print("--- 阶段二: 开始精筛候选集 ---")
start_time_s2 = time.time()  # 记录精筛阶段的开始时间

try:
    # 从上一步生成的 CSV 文件加载候选集
    # low_memory=False 可以稍微加快读取速度，但会占用更多内存，对于候选集通常是值得的
    df_candidates = pd.read_csv(CANDIDATES_FILE, low_memory=False)
    print(f"✅ 成功加载 {len(df_candidates)} 篇候选文章。")
except FileNotFoundError:
    print(f"❌ 错误: 候选文件未找到 {CANDIDATES_FILE}。请先运行优化后的块 3。")
    df_candidates = pd.DataFrame()  # 创建空DataFrame以避免后续代码出错

if not df_candidates.empty:
    texts = df_candidates[NEWS_COLUMN].astype(str)

    results = []
    # 使用 nlp.select_pipes 来临时启用需要的组件，这是一个管理计算资源的好习惯
    with nlp.select_pipes(enable=["parser"]):
        # 批量处理文本，nlp.pipe 会自动利用多核优势
        # batch_size 可以根据你的 CPU 核心数和内存来调整
        docs = nlp.pipe(texts, batch_size=50)

        # 使用 tqdm 包装以显示进度
        for doc in tqdm(docs, total=len(df_candidates), desc="精筛文章"):
            rejection_reason = ""
            is_rejected = False

            # 1. 【核心优化】使用 PhraseMatcher 高效查找关键词
            # matcher(doc) 会在 C 语言层面以极高速度完成所有关键词的查找
            matches = matcher(doc)

            # 如果没有匹配到任何关键词，直接拒绝 (这通常是健全性检查)
            if not matches:
                results.append({'keep': False, 'rejection_reason': '未找到任何关键词(精筛阶段)'})
                continue

            # 2. 【核心优化】从匹配结果中高效地收集信息
            # 使用集合推导式，快速获取所有匹配到的、不重复的关键词文本
            found_keywords_text = {doc[start:end].text.lower() for match_id, start, end in matches}

            # 使用列表推导式，快速从查找字典中获取关键词的详细信息
            found_keywords_info = [keyword_lookup[kw] for kw in found_keywords_text if kw in keyword_lookup]

            # --- 3. 依次应用否决规则 (这部分逻辑保持不变) ---
            is_rejected, rejection_reason = check_low_tier_only(found_keywords_info)

            if not is_rejected:
                is_rejected, rejection_reason = check_negation(doc, found_keywords_text)

            if not is_rejected:
                is_rejected, rejection_reason = check_hypothetical(doc, found_keywords_text)

            # ... 您可以在这里加入更多未来的规则 ...

            results.append({
                'keep': not is_rejected,
                'rejection_reason': rejection_reason
            })

    # --- 合并与保存结果 ---
    print("\n正在合并精筛结果...")
    df_results = pd.DataFrame(results, index=df_candidates.index)
    df_final_with_reasons = pd.concat([df_candidates, df_results], axis=1)

    # 分离通过和被拒绝的文章
    df_accepted = df_final_with_reasons[df_final_with_reasons['keep'] == True].drop(
        columns=['keep', 'rejection_reason'])
    df_rejected = df_final_with_reasons[df_final_with_reasons['keep'] == False].drop(columns=['keep'])

    print("\n--- 精筛完成 ---")
    # 将最终结果保存为 CSV
    df_accepted.to_csv(FINAL_RESULT_FILE, index=False, encoding='utf-8')
    print(f"✅ {len(df_accepted)} 篇最终文章已保存到: {FINAL_RESULT_FILE}")

    df_rejected.to_csv(REJECTED_FILE, index=False, encoding='utf-8')
    print(f"ℹ️ {len(df_rejected)} 篇被拒绝的文章已保存到: {REJECTED_FILE} (供分析)")

    end_time_s2 = time.time()
    print(f"阶段二耗时: {(end_time_s2 - start_time_s2) / 60:.2f} 分钟。")

else:
    print("候选集为空，无需精筛。")

print("\n✅ 块 5: 精筛流程执行完毕。")


--- 阶段二: 开始精筛候选集 ---
✅ 成功加载 179380 篇候选文章。


精筛文章: 100%|██████████| 179380/179380 [35:50<00:00, 83.42it/s] 



正在合并精筛结果...

--- 精筛完成 ---
✅ 178273 篇最终文章已保存到: data_processed/final_china_news.csv
ℹ️ 1107 篇被拒绝的文章已保存到: data_processed/china_news_rejected_articles.csv (供分析)
阶段二耗时: 36.07 分钟。

✅ 块 5: 精筛流程执行完毕。
